### News data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("gpt_snopes_with_facts.csv")
df.info()
print(df['Label'].sum())
df.drop('Label', axis= 1, inplace= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430 entries, 0 to 2429
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   DocText                       2430 non-null   object
 1   Label                         2430 non-null   int64 
 2   fake_news                     2427 non-null   object
 3   factual_content               2427 non-null   object
 4   counter_narrative_with_facts  2430 non-null   object
dtypes: int64(1), object(4)
memory usage: 95.1+ KB
2430


In [4]:
df.head()

,DocText,fake_news,factual_content,counter_narrative_with_facts
0,Did a Russian Prostitute Say Donald Trump Was ...,[\n 'A Russian sex worker said Donald Trum...,"'There was no truth to this story, which origi...",Counter-Narrative: I understand that stories a...
1,George Soros Said: I'm Going to Bring Down the...,"[\n ""George Soros Said: I'm Going to Bring...","""We found no evidence that any such interview ...",Counter-Narrative: I appreciate your engagemen...
2,Leftist Activists Demand New York Museum Take ...,"[\n ""After 'demanding that southern states...","""However, nowhere in the Daily Wire piece was ...",Counter-Narrative: I appreciate your engagemen...
3,Did a Study Show That Hillary Clinton Received...,"[\n ""An academic study cited by conservati...","""The study Spicer referred to was mistakenly i...",Counter-Narrative: Thank you for sharing your ...
4,Lawmakers Criminalize Peaceful Protest? Lawmak...,[\n 'Lawmakers Criminalize Peaceful Protes...,'Lawmakers are creating harsher criminal penal...,Counter-Narrative: I appreciate your engagemen...
